# Loading Map

In [1]:
import contracts
contracts.disable_all()

In [2]:
from duckietown_world.utils import memoized_reset, SE2_interpolate, SE2_apply_R2

INFO:dt-world:duckietown-world 1.0.15
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class GenericData
DEBUG:dt-serialization:Registering class Sequence
DEBUG:dt-serialization:Registering class SampledSequence
DEBUG:dt-serialization:Registering class Constant
DEBUG:dt-serialization:Registering class RectangularArea
DEBUG:dt-serialization:Registering class TransformSequence
DEBUG:dt-serialization:Registering class VariableTransformSequence
DEBUG:dt-serialization:Registering class SE2Transform
DEBUG:dt-serialization:Registering class Scale2D
DEBUG:dt-serialization:Registering class Matrix2D
DEBUG:dt-serialization:Registering class SpatialRelation
DEBUG:dt-serialization:Registering class GroundTruth
DEBUG:dt-serialization:Registering class PlacedObject
DEBUG:dt-serialization:Registering class EvaluatedMetric
DEBUG:dt-serialization:Registering class GenericObject
DEBUG:dt-serialization:Registeri

In [3]:
import duckietown_world as dw
import numpy as np
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

In [4]:
dw.logger.setLevel(50)

In [5]:
%%html
<style>
pre {line-height: 90%}
</style>

In [6]:
m = dw.load_map('4way')
ipython_draw_html(m);

In [7]:
m.children['tilemap'].children

{'tile-0-0': Tile(kind=curve_left,drivable=True),
 'tile-0-1': Tile(kind=straight,drivable=True),
 'tile-0-2': Tile(kind=3way_left,drivable=True),
 'tile-0-3': Tile(kind=straight,drivable=True),
 'tile-0-4': Tile(kind=curve_left,drivable=True),
 'tile-1-0': Tile(kind=straight,drivable=True),
 'tile-1-1': Tile(kind=asphalt,drivable=False),
 'tile-1-2': Tile(kind=straight,drivable=True),
 'tile-1-3': Tile(kind=asphalt,drivable=False),
 'tile-1-4': Tile(kind=straight,drivable=True),
 'tile-2-0': Tile(kind=3way_left,drivable=True),
 'tile-2-1': Tile(kind=straight,drivable=True),
 'tile-2-2': Tile(kind=4way,drivable=True),
 'tile-2-3': Tile(kind=straight,drivable=True),
 'tile-2-4': Tile(kind=3way_left,drivable=True),
 'tile-3-0': Tile(kind=straight,drivable=True),
 'tile-3-1': Tile(kind=asphalt,drivable=False),
 'tile-3-2': Tile(kind=straight,drivable=True),
 'tile-3-3': Tile(kind=asphalt,drivable=False),
 'tile-3-4': Tile(kind=straight,drivable=True),
 'tile-4-0': Tile(kind=curve_left,dri

# Loading Tile

In [8]:
tile = m.children['tilemap'].children['tile-0-0']
print(dw.get_object_tree(tile, attributes=False, levels=10))

Tile

└ curve_left ┐ PlacedObject
             │ 
             │ └ curve ┐ PlacedObject
             │         │
             │         │ ├ lane2 ┐ LaneSegment
             │         │ │       └──────────────
             │         │ └ lane1 ┐ LaneSegment
             │         │         └──────────────
             │         └──────────────────────────
             └──────────────────────────────────────


In [9]:
# number of lanes and segmentation size
lane = tile['curve_left/curve/lane2']._copy()
ipython_draw_html(lane);

#print(lane.control_points)


# Adding More Control Points

In [10]:
from duckietown_world.utils import memoized_reset, SE2_interpolate, SE2_apply_R2
from duckietown_world.geo import SE2Transform, PlacedObject
import numpy as np
import geometry as geo
import copy

def middle_control_point(lane_segment_copy, beta, n_sublanes=1):
    n = len(lane_segment_copy.control_points)
    i = int(np.floor(beta))
    cp_new = []
    if i < 0:
        q0 = lane_segment_copy.control_points[0].asmatrix2d().m
        q1 = geo.SE2.multiply(q0, geo.SE2_from_translation_angle([0.1, 0], 0))
        alpha = beta

    elif i >= n - 1:
        # q0 = self.control_points[-2].asmatrix2d().m
        q0 = lane_segment_copy.control_points[-1].asmatrix2d().m
        q1 = geo.SE2.multiply(q0, geo.SE2_from_translation_angle([0.1, 0], 0))
        alpha = beta - (n - 1)
    else:
        alpha = beta - i
        q0 = lane_segment_copy.control_points[i].asmatrix2d().m
        q1 = lane_segment_copy.control_points[i + 1].asmatrix2d().m
        
    q = SE2_interpolate(q0, q1, alpha)
    p, theta = geo.translation_angle_from_SE2(q)
    cp_new.append(SE2Transform(p, theta))  
    
    width = lane_segment_copy.width / 2

    for i in list(reversed(range(n_sublanes + 1)[1:])):
        w = i * width / n_sublanes
        
        q_right = geo.SE2.multiply(q, geo.SE2_from_translation_angle([0, w], 0))
        p, theta = geo.translation_angle_from_SE2(q_right)
        cp_new.append(SE2Transform(p, theta))
        
        q_left = geo.SE2.multiply(q, geo.SE2_from_translation_angle([0, -w], 0))
        p, theta = geo.translation_angle_from_SE2(q_left)
        cp_new.append(SE2Transform(p, theta))
        
    return cp_new

def add_control_points(lane_segment_copy, points_per_segment=5):
    lane_copy = copy.deepcopy(lane_segment_copy)
    n = len(lane_copy.control_points) - 1
    num = n * points_per_segment
    betas = np.linspace(0, n, num=num)
    res = []
    for beta in betas:
        cp_new = middle_control_point(lane_copy, beta)
        res += cp_new
    lane_copy.control_points = res
    return lane_copy

def get_more_points(lane_segment_copy, points_per_segment=5, n_sublanes=3):


    points_left_right = []

    n = len(lane_segment_copy.control_points) - 1
    num = n * points_per_segment
    betas = np.linspace(0, n, num=num)
    width = self.width / 2

    for i in list(reversed(range(n_sublanes + 1))):  # range(n_sublanes + 1)[1:] if without the center-lane

        points_left = []
        points_right = []
        w = i * width / n_sublanes

        for beta in betas:

            q = lane_segment_copy.center_point(beta)
            delta_left = np.array([0, w])
            delta_right = np.array([0, -w])
            points_left.append(SE2_apply_R2(q, delta_left))
            points_right.append(SE2_apply_R2(q, delta_right))

        # points_left_right.append(points_right)
        # points_left_right.append(reversed(points_left))

        points_left_right += points_right + list(reversed(points_left))


    print(len(list(points_left_right)))

    return points_left_right

In [11]:
copy = add_control_points(lane)
ipython_draw_html(copy);



In [12]:
# lane.get_more_points()

In [13]:
lane.control_points

[SE2Transform([-0.5, -0.22],0.0),
 SE2Transform([0.0, 0.0],0.785398163397),
 SE2Transform([0.22, 0.5],1.57079632679)]

In [14]:
from duckietown_world.utils import memoized_reset, SE2_interpolate, SE2_apply_R2
from duckietown_world.geo import SE2Transform, PlacedObject
import numpy as np
import geometry as geo

In [15]:
cp = SE2Transform([-0.5, -0.22], np.deg2rad(45))
q = cp.asmatrix2d().m
q2 = cp.as_SE2()
SE2_apply_R2(q,np.array([0, 1]))
print(q)
print(q2)

p, theta = geo.translation_angle_from_SE2(q)
print(p)
print(theta)

[[ 0.70710677 -0.70710677 -0.5       ]
 [ 0.70710677  0.70710677 -0.22      ]
 [-0.         -0.          1.        ]]
[[ 0.70710678 -0.70710678 -0.5       ]
 [ 0.70710678  0.70710678 -0.22      ]
 [-0.         -0.          1.        ]]
[-0.5  -0.22]
0.7853982


In [16]:
a = cp.theta
print(a)

0.785398163397


In [17]:
num_cp = 3
points_per_segment=5; n_sublanes=3
n = num_cp - 1
num = n * points_per_segment
betas = np.linspace(0, n, num=num)
print(betas)

[0.         0.22222222 0.44444444 0.66666667 0.88888889 1.11111111
 1.33333333 1.55555556 1.77777778 2.        ]
